In [1]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

import time as time
import platform as platform

import h5py

In [2]:
dir_sep = '/'

In [3]:
from tools.misc_tools import compute_lyapunov_spectrum
from tools.lyapspec_computation_func import create_Kol_data_for_lyap, Kol2D_odd

In [4]:
print(os.getcwd())

/home/rkaushik/Documents/Thesis/MLROM/Kolmogorov


In [5]:
FTYPE = np.float64
ITYPE = np.int64

In [6]:
from numpy import *

data_dir_idx = '001'

dir_name_data = os.getcwd() + dir_sep + 'saved_data' + dir_sep + 'data_' + data_dir_idx
    
with open(dir_name_data + dir_sep + 'sim_data.txt') as f:
    lines = f.readlines()
params_dict = eval(''.join(lines))
Re = params_dict['Re']
delta_t = params_dict['dTr']

with h5py.File(dir_name_data + '/data.h5', 'r') as f:
    print('f.keys() : ', f.keys())
    print('')
    
    N = int(0.5*(np.array(f['num_wavenumbers'])-1))
    print('    N : ', N, type(N))
    N_ref = int(np.array(f['N_ref']))
    print('N_ref : ', N_ref, type(N_ref))
    dt = float(np.array(f['dt']))
    print('   dt : ', dt, type(dt))
    Re = float(np.array(f['Re']))
    print('   Re : ', Re, type(Re))
    
    u_ref = np.array(f['u_reference'], dtype=FTYPE)
    v_ref = np.array(f['v_reference'], dtype=FTYPE)    

f.keys() :  <KeysViewHDF5 ['Dissipation', 'N_ref', 'Re', 'dTr', 'dt', 'kx', 'num_wavenumbers', 't', 'u_reference', 'uh', 'v_reference', 'vh', 'x', 'xx', 'yy']>

    N :  16 <class 'int'>
N_ref :  50 <class 'int'>
   dt :  0.01 <class 'float'>
   Re :  30.0 <class 'float'>


In [7]:
data_meannorm = np.linalg.norm( u_ref-np.mean(u_ref, axis=0), axis=0 )**2
data_meannorm += np.linalg.norm( v_ref-np.mean(v_ref, axis=0), axis=0 )**2
data_meannorm = np.mean( data_meannorm**0.5 ) / u_ref.shape[0]**0.5

In [8]:
init_state_mat = np.empty(shape=(2,)+tuple(u_ref.shape[1:]))
print('init_state_mat.shape : ', init_state_mat.shape)

init_state_mat[0, :, :] = u_ref[-1]
del(u_ref)
init_state_mat[1, :, :] = v_ref[-1]
del(v_ref)

init_state_mat = np.reshape(init_state_mat, (1, -1))
print('init_state_mat.shape : ', init_state_mat.shape)

init_state_mat.shape :  (2, 50, 50)
init_state_mat.shape :  (1, 5000)


In [9]:
n = 4 # forcing frequency

In [10]:
params_mat = np.array([[Re]])

In [11]:
u0h = np.empty(shape=(2*N+1, 2*N+1), dtype=np.complex128)
v0h = np.empty(shape=(2*N+1, 2*N+1), dtype=np.complex128)

du0h_1 = np.empty_like(u0h)
du0h_2 = np.empty_like(u0h)
du0h_3 = np.empty_like(u0h)
du0h_4 = np.empty_like(u0h)
dv0h_1 = np.empty_like(v0h)
dv0h_2 = np.empty_like(v0h)
dv0h_3 = np.empty_like(v0h)
dv0h_4 = np.empty_like(v0h)
midRK_uh = np.empty_like(u0h)
midRK_vh = np.empty_like(v0h)

uh_reference = np.zeros( (N_ref, N_ref), dtype=np.complex128 )
vh_reference = np.zeros( (N_ref, N_ref), dtype=np.complex128 )

In [12]:
kol2D_obj = Kol2D_odd(Re, n, N) 
cdf_kwargs = {
    'T':200,
    't0':0,
    'kol2D_obj':kol2D_obj,
    'delta_t':dt,
    'N_ref':N_ref,
    'N':N,
    'u0h':u0h,
    'v0h':v0h,
    'du0h_1':du0h_1,
    'du0h_2':du0h_2,
    'du0h_3':du0h_3,
    'du0h_4':du0h_4,
    'dv0h_1':dv0h_1,
    'dv0h_2':dv0h_2,
    'dv0h_3':dv0h_3,
    'dv0h_4':dv0h_4,
    'midRK_uh':midRK_uh,
    'midRK_vh':midRK_vh,
    'uh_reference':uh_reference,
    'vh_reference':vh_reference,
    'FTYPE':np.float64,
    'ITYPE':np.int64,
}

create_data_fn = create_Kol_data_for_lyap
num_modes = init_state_mat.shape[1]
num_cases = 1
num_exp = 20 # number of exponents to compute

dy = 1e-7
dy_mat = np.empty(shape=num_cases, dtype=FTYPE)
begin_idx = 0
for i in range(num_cases):
    dy_mat[i] = dy*data_meannorm
    print('dy_mat[{}] :'.format(i), dy_mat[i])
print('')


lpspectrum_mat, _ = compute_lyapunov_spectrum(
    create_data_fn, cdf_kwargs, num_modes, 
    init_state_mat, params_mat, dy_mat,
    zeta=50, delta_completionratio=0.1, num_exp=num_exp, print_flag=True)

dy_mat[0] : 3.876916689795284e-08

number of evaluation intervals per parameter case: 400

case 1 completion_ratio : 0.1, elapsed_time : 405.303s, global_completion : 10.0%
case 1 completion_ratio : 0.2, elapsed_time : 795.703s, global_completion : 20.0%
case 1 completion_ratio : 0.3, elapsed_time : 1180.877s, global_completion : 30.0%
case 1 completion_ratio : 0.4, elapsed_time : 1563.69s, global_completion : 40.0%
case 1 completion_ratio : 0.5, elapsed_time : 1946.808s, global_completion : 50.0%
case 1 completion_ratio : 0.6, elapsed_time : 2330.584s, global_completion : 60.0%
case 1 completion_ratio : 0.7, elapsed_time : 2713.682s, global_completion : 70.0%
case 1 completion_ratio : 0.8, elapsed_time : 3096.053s, global_completion : 80.0%
case 1 completion_ratio : 0.9, elapsed_time : 3470.77s, global_completion : 90.0%
case 1 completion_ratio : 1.0, elapsed_time : 3868.185s, global_completion : 100.0%
case 1 MLE : 0.003639



In [13]:
lpspectrum_mat.sort(axis=1)
lpspectrum_mat = lpspectrum_mat[:, -1::-1]

In [14]:
print(lpspectrum_mat)

[[ 0.00363924  0.00145827 -0.00232293 -0.00307353 -0.01358813 -0.02478011
  -0.0352366  -0.037069   -0.03950068 -0.04603473 -0.06298151 -0.06900153
  -0.07506466 -0.07560256 -0.08442227 -0.0853987  -0.09029686 -0.10205134
  -0.13423373 -0.148958  ]]


In [15]:
print(lpspectrum_mat[:, 0])

[0.00363924]


In [16]:
print(lpspectrum_mat[:, 0]**(-1))

[274.78294464]


In [17]:
lpspectrum_cumsum = np.cumsum(lpspectrum_mat, axis=1)
KY_dim_lst = np.empty(shape=lpspectrum_mat.shape[0])

for i in range(lpspectrum_mat.shape[0]):
    last_idx = np.where(lpspectrum_cumsum[i] >= 0)[0][-1]
    KY_dim = (last_idx+1) + lpspectrum_cumsum[i, last_idx]/np.abs(lpspectrum_mat[i, last_idx+1])
    KY_dim_lst[i] = KY_dim

In [18]:
lpspectrum_cumsum

array([[ 3.63923606e-03,  5.09750613e-03,  2.77457890e-03,
        -2.98951649e-04, -1.38870791e-02, -3.86671877e-02,
        -7.39037846e-02, -1.10972780e-01, -1.50473459e-01,
        -1.96508185e-01, -2.59489695e-01, -3.28491221e-01,
        -4.03555884e-01, -4.79158442e-01, -5.63580710e-01,
        -6.48979410e-01, -7.39276271e-01, -8.41327608e-01,
        -9.75561337e-01, -1.12451933e+00]])

In [19]:
print('KY_dim_lst : ', KY_dim_lst)

KY_dim_lst :  [3.90273347]


In [20]:
np.savez(
    dir_name_data+'/lyapunov_spectrum',
    lyapunov_spectrum_mat=lpspectrum_mat,
    KY_dim_lst=KY_dim_lst
)

In [21]:
with open(dir_name_data+'/lpspec.txt', 'w') as f:
    s = 'lpspectrum : '+str(lpspectrum_mat)
    s += '\n'
    s += 'lp_time : ' + str(lpspectrum_mat[:, 0]**(-1))
    s += '\n'
    s += 'KY_dim : '+str(KY_dim_lst)
    s += '\n'
    f.write(s)